# Goal
Since the beginning of news there has always been fake news tricking the unsuspecting into believeing lies. With the concept of fake news populaized by Donald Trump I aimed to solve world problems with ultimate goal to create model that can classify rather a article is fake or real. I use NLP processes to to accomplish this mission.


<img src="image/MIT-Fake-News.jpg">



# Table of content

- Hyperlinks
- Imports
- Load Data
- Split Data
- Vectorizing
- Grid Search/ Hyperparameter Tuning
- Saving Best Grid Search
- All Data with Best Estimator/Hyperparameters
- Saving Best Model
- Best Estimator Metrics
- Final Model Metrics



- """File Name Format <Website>_<article_name_Camelcase>_<what it is>.<file_type>"""

# Hyperlinks

- [Initial data set from kaggle](https://www.kaggle.com/c/fake-news/data)
- [Additional Huffinton Articles](https://www.kaggle.com/rmisra/news-category-dataset)
- [Additional New York Times Articles](https://www.kaggle.com/nzalake52/new-york-times-articles)
- [Additional sources of fake news from buzzfeed](https://www.buzzfeednews.com/article/craigsilverman/facebook-fake-news-hits-2018)
- [Reference Material](https://www.kaggle.com/plarmuseau/minimalistic-logistic-ngram-tfidf-lb-0-975)
- [List of satire sites](https://libguides.library.kent.edu/c.php?g=278400&p=1854632)
- [Media bias chart](https://www.adfontesmedia.com/interactive-media-bias-chart/?v=402f03a963ba)
- [List of reliable sites from Forbes](https://www.forbes.com/sites/berlinschoolofcreativeleadership/2017/02/01/10-journalism-brands-where-you-will-find-real-facts-rather-than-alternative-facts/#82973b6e9b5a)

# Imports

In [1]:
import boto3
import boto3.session
import numpy as np
import os
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from sklearn.ensemble import (RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier)
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.base import BaseEstimator
from zipfile import ZipFile

# Load Data

In [2]:
_ = os.path.join('data', 'prepared_data_frames_3.zip') 
with ZipFile(_, 'r') as zip: 
    zip.extractall('data')


_  = os.path.join('data', 'prep_final_df.pkl')
_ = open(_, 'rb')
_ = pickle.load(_)
final_df = _

# [AWS SAGE MAKER CODE]
# Get the final Data Frame
# cred = boto3.Session().get_credentials()
# ACCESS_KEY = cred.access_key
# SECRET_KEY = cred.secret_key
# SESSION_TOKEN = cred.token  ## optional

# s3client = boto3.client('s3', 
#                         aws_access_key_id = ACCESS_KEY, 
#                         aws_secret_access_key = SECRET_KEY, 
#                         aws_session_token = SESSION_TOKEN
#                        )

# bucket='<.....>'
# data_key = '<.....>'
# data_location = 's3://{}/{}'.format(bucket, data_key)

# response = s3client.get_object(Bucket=bucket, Key=data_key)

# body = response['Body'].read()
# final_df = pickle.loads(body)

# Split Data

In [20]:
X = final_df['total']
y = final_df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

Average Number of Non-Zero Elements in Vectorized Articles: 374.308123799546
Percentage of columns containing 0: 0.9988211398955031


# Vectorizing

In [25]:
vectorizer = TfidfVectorizer()
tf_idf_data_train = vectorizer.fit_transform(X_train)
tf_idf_data_test = vectorizer.transform(X_test)

non_zero_cols = tf_idf_data_train.nnz / float(tf_idf_data_train.shape[0])
print("Average Number of Non-Zero Elements in Vectorized Articles: {}".format(non_zero_cols))

percent_sparse = 1 - (non_zero_cols / float(tf_idf_data_train.shape[1]))
print('Percentage of columns containing 0: {}'.format(percent_sparse))

Average Number of Non-Zero Elements in Vectorized Articles: 374.8314962348576
Percentage of columns containing 0: 0.9986589548875883


# Grid Search/ Hyperparameter Tuning

In [ ]:
class DummyEstimator(BaseEstimator):
    def fit(self): pass
    def score(self): pass

# Create a pipeline
pipe = Pipeline([('clf', DummyEstimator())]) # Placeholder Estimator

# Candidate learning algorithms and their hyperparameters
search_space = [{'clf': [LogisticRegression()], # Actual Estimator
                 'clf__penalty': ['l1', 'l2'],
                 'clf__C': np.logspace(0, 4, 10),
                 'clf__max_iter': [100, 200, 300]},
                
                {'clf':[MultinomialNB()], # Actual Estimator
                'clf__alpha': [1e-2, 1e-3, 1.0]}, #Parameters
                
                {'clf': [RandomForestClassifier()],
                 'clf__n_estimators' : [50, 100, 200],
                 'clf__max_depth' : [None, 40, 100]},             
               ]


# Create grid search 
gs = GridSearchCV(pipe, search_space)

In [ ]:
gs.fit(tf_idf_data_train, y_train)

In [ ]:
gs.best_estimator_

# Saving Best Grid Search

In [6]:
model_file_name = 'semi_final_model.pkl'

#Save model
with open(model_file_name, 'wb') as out:
    pickle.dump(gs.best_estimator_, out)

#Read Model
#with open(model_file_name , 'rb') as inp:
#    clf = pickle.load(inp)

#[AWS SAGE MAKER CODE] Upload to s3
#boto3.Session().resource('<Value>').Bucket('<Value>').Object('<Value>').upload_file(model_file_name)

# All Data with Best Estimator/Hyperparameters

In [ ]:
vectorizer = TfidfVectorizer()

tf_idf_data = vectorizer.fit_transform(X)

lr = LogisticRegression(C=59.94842503189409, 
                   class_weight=None, 
                   dual=False,
                   fit_intercept=True, 
                   intercept_scaling=1, 
                   max_iter=100,
                   multi_class='warn', 
                   n_jobs=None, 
                   penalty='l2', 
                   random_state=None,
                   solver='warn', 
                   tol=0.0001, 
                   verbose=0, 
                   warm_start=False)

lr.fit(tf_idf_data, y)

# Saving Best Model

In [2]:
model_file_name = 'final_model.pkl'

#Save model
#with open(model_file_name, 'wb') as out:
#    pickle.dump(lr, out)

#Read Model
with open(model_file_name , 'rb') as inp:
    clf = pickle.load(inp)

#[AWS SAGE MAKER CODE] Upload to s3
#boto3.Session().resource('<Value>').Bucket('<Value>').Object('<Value>').upload_file(model_file_name)

C:\Users\Laurent\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.linear_model.logistic module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.linear_model. Anything that cannot be imported from sklearn.linear_model is now part of the private API.
  warnings.warn(message, FutureWarning)
C:\Users\Laurent\Anaconda3\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.20.3 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [6]:
clf.coef_

array([[-1.8586967 , -0.59820096, -0.10114524, ..., -0.07457475,
        -0.07457475, -0.07457475]])

# Best Estimator Metrics

In [9]:
y_pred_train = gs.best_estimator_.predict(tf_idf_data_train)
y_pred_test = gs.best_estimator_.predict(tf_idf_data_test)

In [ ]:
accuracy_score(y_train, y_pred_train), accuracy_score(y_test, y_pred_test)

In [ ]:
classification_report(y_train, y_pred_train, target_names=[0,1])

In [ ]:
classification_report(y_test, y_pred_test, target_names=[0,1])

# Final Model Metrics

In [ ]:
y_pred = clf.predict(tf_idf_data_train)

In [ ]:
accuracy_score(y, y_pred)

In [ ]:
classification_report(y, y_pred, target_names=[0,1])

- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 

# Experiment

In [32]:
X = final_df['total']
vectorizer = TfidfVectorizer()
tf_idf_data_train = vectorizer.fit_transform(X)

In [5]:
with open('vectorizer.pkl', 'wb') as out:
    pickle.dump(vectorizer, out)

In [7]:
#Read Model
with open('final_model.pkl' , 'rb') as inp:
    clf = pickle.load(inp)

C:\Users\Laurent\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.linear_model.logistic module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.linear_model. Anything that cannot be imported from sklearn.linear_model is now part of the private API.
  warnings.warn(message, FutureWarning)
C:\Users\Laurent\Anaconda3\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.20.3 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
